In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\qml.Sample\OptimiseParams\tol"
potential = 'DW'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])
    #abs_tolerance.append(optimzer['abs_tolerance'])
    strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [3]:
df = pd.DataFrame({
    'folder': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    #'abs_tolerance': abs_tolerance,
    'strategy': strategy,
    'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [4]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,tolerance,strategy,popsize,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
3,1e-06,100,10000,1.000000e-06,randtobest1bin,5,100/40,10000,104,3.55950e+00,9.55988e-01,8.91599e-01,1:25:14.432020
1,0.001,100,10000,1.000000e-03,randtobest1bin,5,100/40,10000,100,3.49031e+00,7.09954e-01,8.91599e-01,1:25:09.551523
2,1e-05,100,10000,1.000000e-05,randtobest1bin,5,100/40,10000,111,3.46773e+00,7.46137e-01,8.91599e-01,1:34:41.199623
4,1e-07,100,10000,1.000000e-07,randtobest1bin,5,100/40,10000,104,3.45101e+00,7.67066e-01,8.91599e-01,1:29:10.194740
0,0.0001,100,10000,1.000000e-04,randtobest1bin,5,100/40,10000,106,2.55678e+00,8.04966e-01,8.91599e-01,1:23:25.342813
5,1e-08,100,10000,1.000000e-08,randtobest1bin,5,100/40,10000,106,2.53047e+00,9.03528e-01,8.91599e-01,1:32:13.886910


In [5]:
from tabulate import tabulate
df['folder'] = df['folder'].astype(float)
df = df.drop(columns=['tolerance','num_vqe','shots','maxiter']).sort_values('folder',ascending=False)

latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{rlrlrrrrl}
\hline
   folder & strategy       &   popsize & Converged Runs   &   Mean Iter &   Delta min E &   Delta median E &    Exact & Time           \\
\hline
   0.1    & randtobest1bin &        20 & 40/40            &         104 &   0.00390026  &      0.0593796   & 0.891599 & 0:07:35.734885 \\
   0.01   & randtobest1bin &        20 & 40/40            &         143 &   7.31637e-14 &      4.15631e-05 & 0.891599 & 0:10:12.144011 \\
   0.001  & randtobest1bin &        20 & 40/40            &         296 &   7.31637e-14 &      4.12383e-05 & 0.891599 & 0:19:15.423398 \\
   0.0001 & randtobest1bin &        20 & 40/40            &         414 &   7.31637e-14 &      4.12586e-05 & 0.891599 & 0:26:09.780574 \\
   1e-05  & randtobest1bin &        20 & 39/40            &         611 &   7.31637e-14 &      4.12586e-05 & 0.891599 & 0:34:24.340250 \\
\hline
\end{tabular}
